We test the integration functionality with different element types.
First we import the libraries needed

In [1]:
import meshio as mio
import numpy as np
from mesh.npmesh import NPMesh

\section{First order bases}
The following imports an example mesh containing three squares. Leftmost with quadrangle elements, centermost mixed quadrangles and triangles and rightmost just triangles. Triangles contain three nodes, quadrangles four.

In [2]:
#simple first order mesh
out = mio.read('../npyfem/resources/beast_2d.msh')
msh = NPMesh(*out)

Now let us take a brief look at the element structure of msh.

In [3]:
msh.elementgroups['line']
msh.elementgroups['quad']

We are now interested in region 1, which contains the whole domain. We can see that region 0 contains not only lines (element group 1) but also triangles (element group 2) and quadrangles (element group 3).

We try to integrate surface areas. Let us compute the integral
    $$\int_{\Omega_i} \sqrt{\det g_i}, $$
where $\Omega_i$ is the $i$th reference element (triangle or quadrangle, does not matter too much) and $g_i=J_i^TJ_i$, where $J_i$ is the jacobian of the $i$th element coordinate mapping. We begin with one integration point. Note that $\sqrt{\det g}=\sqrt{\det (J^TJ)}=\sqrt{\det J^T \det J}=| \det J |$ for a square matrix  $J$. But since the mesh is embedded in higher dimension, it is convenient to use the square root of the determinant of $g$ instead.

Next we call for the triangles and quadrangles and define integration by one integration point. Then we define the shape functions in the reference elements.

In [4]:
#call for triangles and make the definitions
triangles = msh.elementgroups['triangle']
triangles.def_integration(1)

In [5]:
#call for quadrangles and make the definitions
quadrangles = msh.elementgroups['quad']
quadrangles.def_integration(1)

The integrations are carried out as follows.

In [6]:
#triangle surfaces over the domain
J_tri = triangles.get_jacobians_isop(triangles.intpoints, [0,1])
JT_tri = J_tri.transpose(0,1,3,2)
g_tri = np.matmul(JT_tri,J_tri)
detg_tri = np.linalg.det(g_tri)
ints_tri = triangles.integrate_ref(np.sqrt(detg_tri))

In [7]:
#quadrangle surfaces over the domain
J_quad = quadrangles.get_jacobians_isop(quadrangles.intpoints, [0,1])
JT_quad = J_quad.transpose(0,1,3,2)
g_quad = np.matmul(JT_quad,J_quad)
detg_quad = np.linalg.det(g_quad)
ints_quad = quadrangles.integrate_ref(np.sqrt(detg_quad))

Now the arrays \emph{ints_tri} and \emph{ints_quad} contain integrals of each element in the corresponding element group. We obtain the surface areas by summing over each index.

In [8]:
ints_tri.sum()+ints_quad.sum()

3.0

The result is the desired one and independent of the number of integration points. Next we compute the line length of the boundary of region 1. This is found in region 2. The integral is just the same 
    $$\int_{\Gamma_i} \sqrt{\det g_i}, $$
where $\Gamma_i$ is the $i$th reference line element and $g_i=J_i^TJ_i$, where $J_i$ is the jacobian of the $i$th line element coordinate mapping. We obtain

In [9]:
lines=msh.elementgroups['line']
lines.def_integration(2)

J_line=lines.get_jacobians_isop(lines.intpoints,[0])
JT_line=J_line.transpose(0,1,3,2)
g_line=np.matmul(JT_line,J_line)
detg_line=np.linalg.det(g_line)
ints_line=lines.integrate_ref(np.sqrt(detg_line))
ints_line.sum()

5.9999999999999982

which is the desired result.

# Higher order bases
The following imports the same mesh containing three squares. This time the triangles contain six nodes, quadrangles nine and the lines two. Surface areas are computed as

In [10]:
#simple first order mesh
out = mio.read('../npyfem/resources/beast_2d_2ord.msh')
msh = NPMesh(*out)

In [11]:
#call for triangles and make the definitions
triangles=msh.elementgroups['triangle6']
triangles.def_integration(3)

#call for quadrangles and make the definitions
quadrangles=msh.elementgroups['quad9']
quadrangles.def_integration(4)

#triangle surfaces over the domain
J_tri=triangles.get_jacobians_isop(triangles.intpoints, [0,1])
JT_tri=J_tri.transpose(0,1,3,2)
g_tri=np.matmul(JT_tri,J_tri)
detg_tri=np.linalg.det(g_tri)
ints_tri=triangles.integrate_ref(np.sqrt(detg_tri))

#quadrangle surfaces over the domain
J_quad=quadrangles.get_jacobians_isop(quadrangles.intpoints, [0,1])
JT_quad=J_quad.transpose(0,1,3,2)
g_quad=np.matmul(JT_quad,J_quad)
detg_quad=np.linalg.det(g_quad)
ints_quad=quadrangles.integrate_ref(np.sqrt(detg_quad))

ints_tri.sum()+ints_quad.sum()

2.9999999999999996

and line lengths as

In [12]:
lines=msh.elementgroups['line3']
lines.def_integration(2)

J_line=lines.get_jacobians_isop(lines.intpoints, [0])
JT_line=J_line.transpose(0,1,3,2)
g_line=np.matmul(JT_line,J_line)
detg_line=np.linalg.det(g_line)
ints_line=lines.integrate_ref(np.sqrt(detg_line))
ints_line.sum()

6.0

who indeed are the same in all ways.